In [1]:
from datetime import date
from datetime import datetime
from koboextractor import KoboExtractor
import pandas as pd

from math import pi
import pandas as pd
from bokeh.palettes import Category20c
from bokeh.plotting import figure, show
from bokeh.transform import cumsum

import bokeh
from bokeh.io import output_file, show
from bokeh.layouts import row,column
from bokeh.plotting import figure,output_file, save, show, ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.resources import CDN
from bokeh.embed import components

from collections import OrderedDict
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
# Obtener los datos de la base de datos de internet
KOBO_TOKEN = '26566dd7934bcb45fee86851765c4f7c867e9cc3'
kobo = KoboExtractor(KOBO_TOKEN, 'https://kf.kobotoolbox.org/api/v2') # kobo.humanitarianresponse.info
asset_uid = "ajYQ63QAQhzKx7F3F9zEvR" #"a7Wd6TUVqejWxSwBkzSGY9"
# asset_uid = "aGigj5YaeQpmeyqvzjUEVp" Para el otro formulario
#today = date.today()
#datos_today = kobo.get_data(asset_uid,submitted_after = today )
data = kobo.get_data(asset_uid)
data = kobo . sort_results_by_time(data['results'], reverse = True) 
#data = data['results']

puntos = ["punto_1", "punto_2", "punto_3", "punto_4", "punto_5" , "punto_6"]
data

[{'_id': 118358949,
  'formhub/uuid': 'b142669bf51e453495b1709df7d1aea8',
  'start': '2021-10-09T15:41:07.637-03:00',
  'end': '2021-10-09T15:42:32.439-03:00',
  'Nombre': 'Anonnimo',
  'Observador': 'turista',
  'Pais': 'CHILE',
  'Fecha_y_Hora': '2021-10-09T15:42:00-03:00',
  'Punto_de_Observaci_n': 'punto_3',
  'Subir_Imagen': '16338049367711412659171-15_42_26.jpg',
  '__version__': 'vBxxVut85VBR8KKB3QBwgw',
  '_version_': 'vLc6HEZxnAnnhNPkeTRSvx',
  '_version__001': 'vHgopGkQ74T6PHF4N7g88Q',
  'meta/instanceID': 'uuid:245c2f89-de49-4e1d-91f9-33915c755a3f',
  '_xform_id_string': 'ajYQ63QAQhzKx7F3F9zEvR',
  '_uuid': '245c2f89-de49-4e1d-91f9-33915c755a3f',
  '_attachments': [{'download_url': 'https://kc.kobotoolbox.org/media/original?media_file=marcosgomez%2Fattachments%2Fb142669bf51e453495b1709df7d1aea8%2F245c2f89-de49-4e1d-91f9-33915c755a3f%2F16338049367711412659171-15_42_26.jpg',
    'download_large_url': 'https://kc.kobotoolbox.org/media/large?media_file=marcosgomez%2Fattachments%

In [3]:
# Obtener las imagenes
def getUrlData(punto,data,tipo):
    lista_data_i = []
    #lista_name = []
    j = 0
    for i in range (len(data)):
        if data[i][tipo] == punto:
            lista_data_i.append(data[i]['_attachments'][0]['download_url'])
            #lista_name.append(data[i][])
            j =+ 1
            
        if j > 3 :
            break
            
    return lista_data_i

In [4]:
# Obtener datos de paises 
def getCountryData(punto,data,tipo):
    dict_data = {}
    for i in range (len(data)):
        if data[i][tipo] == punto:
            try:
                dict_data[data[i]['Pais']] = dict_data[data[i]['Pais']] + 1
            except:
                x = {data[i]['Pais'] : 1}
                dict_data.update(x)
                
    return dict_data

In [5]:
def getGraphicCountry(punto,x):
    
    data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'country'})
    #display(x)
    
    p = figure(x_range=data['country'].tolist(), height=350, title="Nacionalidades/Nationalities",
           toolbar_location=None, sizing_mode='scale_width')
    
    p.vbar(x=data['country'].tolist(), top=data['value'].tolist(), width=0.9)
    
    p.xgrid.grid_line_color = None
    p.y_range.start = 0

    script , div = components(p)
    #show(p)
    
    return script , div

## codigo para el gif de las imagenes.

In [6]:
import imageio
import cv2
import urllib
import numpy as np
import requests
from io import StringIO, BytesIO
from PIL import Image

def getUrlDataList(punto, data, tipo):
    lista_data_i = []
    lista_name = []
    for i in range (len(data)):
        if data[i][tipo] == punto:
            lista_data_i.append(data[i]['_attachments'][0]['download_medium_url'])
            lista_name.append(data[i]['Nombre'])
    
    #display(lista_name)
    
    img_array = []
    for x, y in zip(lista_data_i, lista_name):
        
        #Asignar a variable leer_imagen, el nombre de cada imagen
        leer_imagen = imageio.imread(x)
        
        
        #req = requests.get(x,headers=header)
        #display(req.content.decode())
        #req = urllib.request.urlopen(x)
        #arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
        #leer_imagen = cv2.imdecode(arr, -1)
        #$display(BytesIO(req.content))
        #img = Image.open(StringIO(BytesIO(req.content)))
        #leer_imagen = imageio.imread(BytesIO(req.content) + ".jpg")
        #arr = np.asarray(req, dtype=np.uint8)
        #leer_imagen = cv2.imdecode(arr, -1)
        
        texto = y
        posicion = (100,100)
        font = cv2.FONT_ITALIC
        tamaño = 2
        colorLetra = (250,250,250)
        grosorLetra = 5
        
        cv2.putText(leer_imagen,texto,posicion,font,tamaño,colorLetra,grosorLetra)
    
        # añadir imágenes al arreglo img_array
        img_array.append(leer_imagen)
        
    
    #Guardar Gif
    nombre_archivo = punto + '.gif'
    try:
        imageio.mimwrite(nombre_archivo, img_array, 'GIF', duration=1)
    except:
        print("No image")
    

In [8]:
imageio.imread('https://kc.kobotoolbox.org/media/medium?media_file=marcosgomez%2Fattachments%2Fb142669bf51e453495b1709df7d1aea8%2Fcf2b928c-e617-49e1-b7fe-a361bb8e989a%2F16336552240861441456711-22_7_25.jpg')

HTTPError: HTTP Error 403: Forbidden

In [ ]:
import pickle

for punto in puntos:
    
    try:
        script, div = getGraphicCountry(punto,getCountryData(punto,data,'Punto_de_Observaci_n'))
    except:
        print("ERROR: getGraphicCountry => " + punto)
        
    try:
        img = getUrlData(punto, data, 'Punto_de_Observaci_n')
    except:
        print("ERROR: getUrlData => " + punto)
        
    #try:
    getUrlDataList(punto, data, 'Punto_de_Observaci_n') # generar GIF
    #except:
    #    print("ERROR: GIF => " + punto)
    
    
    num_med = 5
    perdido = 10
    
    d = {"punto": punto , "div" : div ,"script" : script,
        "img" : img, 
       "num_mediciones" : num_med, "perdido" : perdido}
    
    #d = {"punto": punto , "div" : div ,"script" : script,
#   "img1" : img_uno , "img2": img_dos ,"img3": img_tres, "img4": img_cuatro,
#       "num_mediciones" : num_med, "perdido" : perdido}
    #df = pd.DataFrame(d, columns = ["punto", "div","script", "img1", "img2","img3", "img4","num_mediciones" , "perdido"], index=[0])
    #df.to_csv('../../static/csv_glaciar/example_cambio.csv')
    
    nombre_archivo = punto + ".json"
    with open(nombre_archivo, 'wb') as fp:
        pickle.dump(d, fp)
    
    #with open(nombre_archivo, 'rb') as fp:
    #    dat = pickle.load(fp)
    #print(dat)

In [ ]:
d = {"punto": punto , "div" : div ,"script" : script,
        "img1" : img_uno , "img2": img_dos ,"img3": img_tres, "img4": img_cuatro,
       "num_mediciones" : num_med, "perdido" : perdido}
df = pd.DataFrame(d, columns = ["punto", "div","script", "img1", "img2","img3", "img4","num_mediciones" , "perdido"], index=[0])
df.to_csv('../../static/csv_glaciar/example_cambio.csv')

# Baliza

In [38]:
# Obtener los datos de la base de datos de internet
KOBO_TOKEN = '26566dd7934bcb45fee86851765c4f7c867e9cc3'
kobo = KoboExtractor(KOBO_TOKEN, 'https://kf.kobotoolbox.org/api/v2') # kobo.humanitarianresponse.info
#asset_uid = "atX9jThsvhvhhjFf573eng" #"a7Wd6TUVqejWxSwBkzSGY9"
asset_uid = "a5xomk6KgJfvM3CpiX23tH" # Para el otro formulario
#today = date.today()
#datos_today = kobo.get_data(asset_uid,submitted_after = today )
data = kobo.get_data(asset_uid)
data = data['results']

#puntos = ["punto_1", "punto_2", "punto_3", "punto_4", "punto_5" , "punto_6"]
baliza = ["b1","b2","b3","b4","b5","b6"]
data

[{'_id': 118080999,
  'formhub/uuid': 'b142669bf51e453495b1709df7d1aea8',
  'start': '2021-10-07T18:10:24.413-03:00',
  'end': '2021-10-07T18:11:28.514-03:00',
  'Nombre': 'Marcos',
  'Observador': 'gu_a',
  'Pais': 'CHILE',
  'Fecha_y_Hora': '2021-10-07T15:15:00.000-03:00',
  'Punto_de_Observaci_n': 'punto_4',
  'Subir_Imagen': 'Balizaa-18_11_16.JPG',
  '__version__': 'vBxxVut85VBR8KKB3QBwgw',
  '_version_': 'vLc6HEZxnAnnhNPkeTRSvx',
  '_version__001': 'vHgopGkQ74T6PHF4N7g88Q',
  'meta/instanceID': 'uuid:baaf6713-1ada-4fce-be74-a38d12145ea6',
  '_xform_id_string': 'ajYQ63QAQhzKx7F3F9zEvR',
  '_uuid': 'baaf6713-1ada-4fce-be74-a38d12145ea6',
  '_attachments': [{'download_url': 'https://kc.kobotoolbox.org/media/original?media_file=marcosgomez%2Fattachments%2Fb142669bf51e453495b1709df7d1aea8%2Fbaaf6713-1ada-4fce-be74-a38d12145ea6%2FBalizaa-18_11_16.JPG',
    'download_large_url': 'https://kc.kobotoolbox.org/media/large?media_file=marcosgomez%2Fattachments%2Fb142669bf51e453495b1709df7d1aea

In [31]:
def generateGraphics(data,punto):
    #Graficando con Bokeh
    source=  ColumnDataSource(ColumnDataSource.from_df(data))
    #Figura width=750, plot_height=250,
        
    fig = figure(width=750, plot_height=280, title=None,
                     x_axis_type = "datetime",
                     tools= 'pan,box_zoom,save,reset,hover', sizing_mode='scale_width')
            
    #Gráfico línea
    fig.line(x="date_time",y="baliza",color='dodgerblue',line_width = 2,
                 legend_label= "Emergencia de baliza(cm)",source=source)
    
    #Puntito sobre cada dato
    fig.circle(x="date_time",
                y="baliza",
               source=source,
               color='orangered',
               selection_color='deepskyblue',
               nonselection_color='lightgray',
                nonselection_alpha=0.3,
              size=8)
        
    #Esto no está funcionando la idea es que se desplieguen estos datos cuando pinchas uno de los puntos
    # Format tooltip
    tooltips = [ ('Pais', '@Pais'),
                    ('Nombre', '@Nombre'),
                    ('Grupo','@Grupo')]

    #tooltips = """
    #    <div>
    #        <h3> Baliza: @Seccion </h3>
    #        <div> <strong>Pais:</strong> @Pais</div>
    #       <div> <strong>Nombre </strong>@Nombre</div>
    #       <div> <strong>Grupo</strong>@Grupo </div>
    #   </div>
    #
    #"""

    #Labels
    fig.xaxis.axis_label = "Fecha"
    fig.yaxis.axis_label = "Pérdida de hielo (cm)"
    #Leyenda
    fig.legend.location = "top_left"
    fig.legend.click_policy="hide"
    fig.legend.padding=5
    fig.legend.margin=0

    #Circulo que aparece al acercarse al dato
    hover_glyph = fig.circle(x='date_time', y='baliza', source=source,
                            size=15, alpha=0,
                           hover_fill_color='orangered', hover_alpha=0.5)

    #Tools
    #fig.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))
    #fig.hover = HoverTool(tooltips=tooltips, renderers=[hover_glyph])

    fig.hover.tooltips = tooltips

    script , div = components(fig)
    #show(fig)
    
    return script , div

In [32]:
def getGraphicsGlaciar(data, punto):
    x = ['Pais','Grupo','cm','Seccion','Nombre','date','time']
    df = pd.DataFrame()
    
    for i in x:
        df[i] = None

    for i in data:
        if i['Baliza'] == punto:
            df = df.append({'Pais' : i['Pa_s'],'Grupo' : i['Observador'],'cm' : int(i['Altura_cms']),'Seccion' : i['Segmento_Baliza'],
                       'Nombre' : i['Observador'],'Fecha' : i['Fecha_y_Hora'][:10],
                            'Hora' : i['Fecha_y_Hora'][len(data[0]['Fecha_y_Hora'])-5:]},
                      ignore_index=True)
    df
    
    #Transformando fecha y hora a datetime y eliminando nans
    df['date']=pd.to_datetime(df['Fecha'], format="%Y-%m-%d").dt.date.astype(str)
    df['time']=pd.to_datetime(df['Hora'], format="%H:%M").dt.time.astype(str)
    df["date_time"]=pd.to_datetime(df['Fecha']+" "+df['Hora'])
    df.set_index('date_time', inplace=True)
    df.drop(["Fecha","Hora"],axis=1,inplace=True)
    df.dropna(inplace=True);
    
    #Sumando largo de balizas anteriores para ver pérdida total de hielo en el periodo
    baliza= []
    for i in range(df.shape[0]):
        if df["Seccion"].iloc[i] == "1":
            baliza.append(df["cm"].iloc[i])
        elif df["Seccion"].iloc[i] == "2":
            baliza.append(df["cm"].iloc[i]+200)
        elif df["Seccion"].iloc[i] == "3":
            baliza.append(df["cm"].iloc[i]+400)
        elif df["Seccion"].iloc[i] == "4":
            baliza.append(df["cm"].iloc[i]+600)
        
    df["baliza"]= baliza
    
    script , div = generateGraphics(df,punto)
    print(div)
    #print(script)
    
    return script , div #df

In [33]:
def numberOfMedition(data, tipoPunto, punto):
    n = 0
    for i in data:
        if i[tipoPunto] == punto:
            n += 1
    #print(n)
    return n

In [34]:
def lostThickness(data):
    c = 0
    return c

In [36]:
#script1 , div1 = getGraphicsGlaciar(data, "b1")
#print(div1)
#print(script1)
#script2 , div2 = getGraphicCountry("b1",getCountryData("b1",data,"Baliza"))

In [37]:
for punto in baliza:
    script1 , div1 = getGraphicsGlaciar(data, punto)
    script2 , div2 = getGraphicCountry("b1",getCountryData(punto,data,"Baliza"))
    img = getUrlData(punto, data, 'Baliza')
    num_med = numberOfMedition(data,"Baliza",punto)
    perdido = lostThickness(data)

KeyError: 'Baliza'

In [17]:
d = {"punto": punto , "div1" : div1 ,"script1" : script1 , "div2" : div2 ,"script2" : script2,
        "img1" : img_uno , "img2": img_dos ,"img3": img_tres, "img4": img_cuatro,
       "num_mediciones" : num_med, "perdido" : perdido}
df = pd.DataFrame(d, columns = ["punto", "div1","script1", "div2","script2", "img1", "img2","img3", "img4","num_mediciones" , "perdido"], index=[0])
df.to_csv('../../static/csv_glaciar/example_baliza.csv')

In [189]:
#df = pd.read_csv("C:/Users/Sebastián-Durán/Documents/P R O Y E C T O S/Nueva carpeta/Geografia/static/csv_glaciar/example_baliza.csv")
#print(df["div1"])
#print(pd.DataFrame.from_dict(df["div1"])["div1"])
#print(df['div1'].values[0])